# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Jan 27th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [1]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd
/data/pt_02586/sub_01/preprocessing


: 1

In [2]:
# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';

# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';


: 1

### QC of eddy

#### option 1: FSL eddyqc tool

The tool eddy_quad from FSL produce a pdf report from a single subject.  
https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddyqc/UsersGuide

It is only available from FSL 6.0 and onward (for instance at the MPI, I have to use this environement "FSL --version 6.0.1")


eddy_quad <eddyBase> -idx <eddyIndex> -par <eddyParams> -m <mask> -b <bvals>  
It takes as input the '--out' param from the eddy command (eddyBase).  
The rest are all the other eddy command input; masks, bval, eddy index (indices) and eddy param (config)  
    
#### If we had the data for topup, we would also need to include the topup estimated field here in the '-f' flag

In [ ]:
eddy_quad dwi_eddy \
    -idx eddy_ind.txt \
    -par eddy_config.txt \
    -m mask_for_eddy.nii.gz \
    -b bvals.txt \
    -v

It creates a folder called dwi_eddy.qc (by default) containing a few things; we only care about the pdf.

The pdf contains graph of subject motion in translation and rotation through time.  
Some graphs of eddy estimated outlier slices (remember that we are NOT using that information here, its just showing what eddy calculated)  
And finaly we have some mean images post eddy.

Once you have run eddy_quad on all subjects,  
The tool eddy_squad can read all the reports from eddy_quad and produce a study level report.

In [ ]:
# First we go back to the root folder
# and create a list of all the eddy reports
# in a new textfile myquads.txt
cd ROOTDIR
for SUBID in {01..21}; 
do 
    echo 'sub_'$SUBID'/preprocessing/dwi_eddy.qc' >> myquads.txt; 
done

# we then run the study level command on the list
eddy_squad myquads.txt

# A folder named "squad" with a pdf is generated by default

# We go back to the processing folder for the rest of the tutorial
cd $SUBJECTDIR'preprocessing'
pwd

#### option 2: Another quick DTI fit

This is identical to what we did earlier (begining of january_20)

In [ ]:
# Like we did before eddy, we do a quick dti fit and FA map
# using the eddy corrected data and the new bvec
dwi2tensor dwi_eddy.nii.gz dti_post_eddy.nii.gz -fslgrad $BVEC $BVAL;
tensor2metric dti_post_eddy.nii.gz -fa fa_post_eddy.nii.gz;

# we can then visually compare it to the quick dti pre eddy.

### New brain mask (post eddy)

Similar to what we did before eddy, we want to generate a new brain mask using our corrected data.  

The first few steps are essentially the same:  
- extract the b0 volumes from the eddy correted data.  
- Take the mean (no need to run motion correction, the output of eddy is already motion corrected).  
- Apply a few iteration of N4 homogenisation (3x).  
- Compute the brain mask with fsl BET at multiple different threshld parameter.  



In [ ]:
# These commands are identical to the ones from january_13
dwiextract $SUBJECTDIR'preprocessing/dwi_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_denoise_degibbs_eddy.nii.gz' -bzero -fslgrad $BVEC $BVAL;
mrmath $SUBJECTDIR'preprocessing/b0_denoise_degibbs_eddy.nii.gz' mean $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy.nii.gz' -axis 3;
N4BiasFieldCorrection -d 3 -i $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy.nii.gz'      -o $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x1.nii.gz';
N4BiasFieldCorrection -d 3 -i $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x1.nii.gz' -o $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x2.nii.gz';
N4BiasFieldCorrection -d 3 -i $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x2.nii.gz' -o $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x3.nii.gz';
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3.nii.gz' -m -n -f 0.3 -R;
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4.nii.gz' -m -n -f 0.4 -R;
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5.nii.gz' -m -n -f 0.5 -R;
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_eddy_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6.nii.gz' -m -n -f 0.6 -R;


In [4]:
MNIFA='/afs/cbs.mpg.de/software/fsl/6.0.3/ubuntu-bionic-amd64/data/standard/FSL_HCP1065_FA_1mm.nii.gz'

ta/standard/FSL_HCP1065_FA_1mm.nii.gz'


: 1

This mask will be used for the registration of our FA to a template in MNI space.  
The template as really sharp brain edges and we need the same level of sharpness in the mask, otherwise it will be miss registered.  

For this reason, we will perform erosion on the BET mask.  
Erosion is an operation that essentially "removes 1 voxel all around" at each pass.  

Here, for each different threshold of BET, we will apply 1, 2 and 3 passes of erosion and choose a good mask from all these options.

In [ ]:
# apply 1, 2 and 3 erosion to all the mask with different threshold
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask_erox1.nii.gz' -npass 1;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask_erox1.nii.gz' -npass 1;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask_erox1.nii.gz' -npass 1;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask_erox1.nii.gz' -npass 1;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask_erox2.nii.gz' -npass 2;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask_erox2.nii.gz' -npass 2;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask_erox2.nii.gz' -npass 2;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask_erox2.nii.gz' -npass 2;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask_erox3.nii.gz' -npass 3;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask_erox3.nii.gz' -npass 3;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask_erox3.nii.gz' -npass 3;
maskfilter $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask.nii.gz' erode $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask_erox3.nii.gz' -npass 3;    

In [ ]:
# Apply all the mask to the FA
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p3.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p4.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p5.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p6.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask_erox1.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p3_erox1.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask_erox1.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p4_erox1.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask_erox1.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p5_erox1.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask_erox1.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p6_erox1.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask_erox2.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p3_erox2.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask_erox2.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p4_erox2.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask_erox2.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p5_erox2.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask_erox2.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p6_erox2.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p3_mask_erox3.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p3_erox3.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p4_mask_erox3.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p4_erox3.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p5_mask_erox3.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p5_erox3.nii.gz';
mrcalc $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' $SUBJECTDIR'preprocessing/b0_eddy_avg_N4x3_bet_0p6_mask_erox3.nii.gz' -mul $SUBJECTDIR'preprocessing/fa_post_eddy_masked_0p6_erox3.nii.gz';


In [3]:
# Visually select the correct mask
# For most subject here it was BET 0.4 and erosion 3 times

# good mask:
# b0_eddy_avg_N4x3_bet_0p4_mask_erox3.nii.gz
# masked FA for registration:
# fa_post_eddy_masked_0p4_erox3.nii.gz

# We can save it to a variable for each subject
BETMASKANTS='b0_eddy_avg_N4x3_bet_0p4_mask_erox3.nii.gz'
MASKEDFAANTS='fa_post_eddy_masked_0p4_erox3.nii.gz'

# Visually select the correct mask
# For most subject here it was BET 0.4 and erosion 3 times

# good mask:
# b0_eddy_avg_N4x3_bet_0p4_mask_erox3.nii.gz
# masked FA for registration:
# fa_post_eddy_masked_0p4_erox3.nii.gz

# We can save it to a variable for each subject
BETMASKANTS='b0_eddy_avg_N4x3_bet_0p4_mask_erox3.nii.gz'
MASKEDFAANTS='fa_post_eddy_masked_0p4_erox3.nii.gz'


: 1

In [ ]:
antsRegistrationSyNQuick.sh -d 3 \
    -f $MNIFA \
    -m $MASKEDFAANTS \
    -o fa_eddy \
    -n 20;

<!-- quicksyn fa to atlas -->
<!-- qc forward and inverse -->

<!-- show and explain true command ants -->

In [ ]:
part 1 of the transform "fa_eddy0GenericAffine.mat"*
part 2 of the transform "fa_eddy1Warp.nii.gz"*
part 2 of the inverse transform "fa_eddy1InverseWarp.nii.gz"*
subject FA in MNIspace "fa_eddyWarped.nii.gz"*
template in subject space "fa_eddyInverseWarped.nii.gz"*
